# Run an experiment script
In the previous example, you ran an experiment inline in this notebook. A more flexible solution is to create a separate script for the experiment, and store it in a folder along with any other files it needs, and then use Azure ML to run the experiment based on the script in the folder.

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("(",compute.status.state ,") ",compute.name, ":", compute.type)

Ready to use Azure ML 1.28.0 to work with aml-workspace
( Stopped )  basic-instance : ComputeInstance


To run the script, you must create a **ScriptRunConfig** that identifies the Python script file to be run in the experiment, and then run an experiment based on it.

> **Note**: The ScriptRunConfig also determines the compute target and Python environment. In this case, the Python environment is defined to include some Conda and pip packages, but the compute target is omitted; so the default local compute will be used.

The following cell configures and submits the script-based experiment.

In [2]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
experiment_env = Environment("diabetes-classification")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
experiment_env.python.conda_dependencies = packages
experiment_env.python.user_managed_dependencies = True


# Create a script config
script_config = ScriptRunConfig(source_directory='../scripts',
                                script='diabetes_logReg.py',
                                arguments = ['--reg-rate', 0.1],
                                environment=experiment_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='diabetes-classification-experiment')
run = experiment.submit(config=script_config)

RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-classification-experiment_1621842903_83aee07e',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-05-24T07:55:10.311966Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '41c77a2f-7260-43b9-aee4-0b8edcb57d9a',
  'azureml.git.repository_uri': 'https://github.com/RohitashwaChaks/dp100.git',
  'mlflow.source.git.repoURL': 'https://github.com/RohitashwaChaks/dp100.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '1e1080875d1a988c9b92431865228ab434d83c85',
  'mlflow.source.git.commit': '1e1080875d1a988c9b92431865228ab434d83c85',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_logReg.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg-rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data

## Current Run Files

In [3]:
# Get output files
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)


Files:
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8252_azureml.log


## Run History

In [4]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['diabetes-classification-experiment']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

Run ID: diabetes-classification-experiment_1621842903_83aee07e
- Accuracy 0.788
Run ID: diabetes-classification-experiment_1621842364_00626ea3
- Accuracy 0.7916666666666666
